In [39]:
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import pandas as pd
import os

In [40]:
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
torch.use_deterministic_algorithms(True, warn_only=True)

os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"

torch.manual_seed(40)
torch.cuda.manual_seed_all(40)


In [41]:
!wget https://www.cse.iitb.ac.in/~pjyothi/cs335/dataset.tar.gz

--2023-10-09 13:37:34--  https://www.cse.iitb.ac.in/~pjyothi/cs335/dataset.tar.gz
Resolving www.cse.iitb.ac.in (www.cse.iitb.ac.in)... 103.21.127.134
Connecting to www.cse.iitb.ac.in (www.cse.iitb.ac.in)|103.21.127.134|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 48946920 (47M) [application/x-gzip]
Saving to: ‘dataset.tar.gz’

dataset.tar.gz      100%[===================>]  46.68M  9.12MB/s    in 5.1s    

2023-10-09 13:37:41 (9.12 MB/s) - ‘dataset.tar.gz’ saved [48946920/48946920]



In [42]:
!mv dataset.tar.gz?rlkey=vajo7g4w8nl1q92ikv8qu75qu dataset.tar.gz

mv: cannot stat 'dataset.tar.gz?rlkey=vajo7g4w8nl1q92ikv8qu75qu': No such file or directory


In [43]:
!tar -xzvf dataset.tar.gz

dataset/
dataset/kaggle/
dataset/kaggle/686.jpg
dataset/kaggle/670.jpg
dataset/kaggle/239.jpg
dataset/kaggle/728.jpg
dataset/kaggle/536.jpg
dataset/kaggle/547.jpg
dataset/kaggle/692.jpg
dataset/kaggle/669.jpg
dataset/kaggle/557.jpg
dataset/kaggle/1.jpg
dataset/kaggle/416.jpg
dataset/kaggle/197.jpg
dataset/kaggle/776.jpg
dataset/kaggle/694.jpg
dataset/kaggle/462.jpg
dataset/kaggle/323.jpg
dataset/kaggle/605.jpg
dataset/kaggle/464.jpg
dataset/kaggle/113.jpg
dataset/kaggle/266.jpg
dataset/kaggle/261.jpg
dataset/kaggle/103.jpg
dataset/kaggle/85.jpg
dataset/kaggle/574.jpg
dataset/kaggle/224.jpg
dataset/kaggle/121.jpg
dataset/kaggle/19.jpg
dataset/kaggle/202.jpg
dataset/kaggle/550.jpg
dataset/kaggle/148.jpg
dataset/kaggle/334.jpg
dataset/kaggle/517.jpg
dataset/kaggle/665.jpg
dataset/kaggle/242.jpg
dataset/kaggle/223.jpg
dataset/kaggle/238.jpg
dataset/kaggle/56.jpg
dataset/kaggle/274.jpg
dataset/kaggle/491.jpg
dataset/kaggle/88.jpg
dataset/kaggle/543.jpg
dataset/kaggle/317.jpg
dataset/kaggle/

In [44]:
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'


In [45]:
def read_image_tensor(image_folder,transform,num_images=None):
    if num_images==None:
        num_images = len(os.listdir(image_folder))
    images = []
    for i in range(num_images):
        img = torchvision.io.read_image(os.path.join(image_folder,f"{i}.jpg")).float()
        images.append(transform(img))
    return torch.stack(images).to(device)



In [46]:
def get_labels(csv_file):
    # TODO: Return a torch tensor after reading the labels in csv_file. Convert to float().
    df = pd.read_csv(csv_file)
    labels_tensor = torch.tensor(df['label'], dtype=torch.float32)
    return labels_tensor.to(device)

In [47]:
img_size = (256,256)
base_transform = transforms.Compose(
    [transforms.Resize(img_size)
    ]
)
train_X = read_image_tensor("dataset/train/",base_transform)/256
train_Y = get_labels("dataset/train.csv")
test_X = read_image_tensor("dataset/test/",base_transform)/256
test_Y = get_labels("dataset/test.csv")

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across th

In [48]:
def train_model(model, train_loader, test_loader, num_epochs, loss_function, optimizer):
    # TODO: Make sure you read through these lines of code and understand all key lines.
    # For example: Why do you need to zero out the gradients using optimizer.zero_grad() in the for loop?
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0.0
        for i, data in enumerate(train_loader):
            inputs,labels = data
            optimizer.zero_grad()
            output = model(inputs)
            loss = loss_function(output,labels.view(output.shape))
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        average_loss = total_loss/len(train_loader)

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {average_loss:.4f}")
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for data in test_loader:
                inputs, labels = data
                outputs = model(inputs)
                pred = (outputs > 0.5)*1
                correct += (pred==labels.view(pred.shape)).sum()
                total += labels.size(0)
            accur = 100*correct/total
            print(f"Test Accuracy after Epoch {epoch+1}: {accur:.2f}%")

    print("Training done.")


In [49]:
# PART 1: TODO
# Write down the model description
# model = ...
# Relevant torch.nn classes you will need include nn.Sequential, nn.Conv2d, nn.MaxPool2d and so on.

class ConvNueralNet(nn.Module):
  def __init__(self):
    super(ConvNueralNet,self).__init__()

    self.conv_layer1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
    self.relu1 = nn.ReLU()
    self.max_pool1 = nn.MaxPool2d(kernel_size = 2, stride = 2)

    self.conv_layer2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
    self.relu2 = nn.ReLU()
    self.max_pool2 = nn.MaxPool2d(kernel_size = 2, stride = 2)

    self.flatten = nn.Flatten()

    self.fc1 = nn.Linear(32*64*64,64)
    self.relu3 = nn.ReLU()
    self.fc2 = nn.Linear(64,1)

    self.sigmoid = nn.Sigmoid()

  def forward(self, x):
      x = self.conv_layer1(x)
      x = self.relu1(x)
      x = self.max_pool1(x)

      x = self.conv_layer2(x)
      x = self.relu2(x)
      x = self.max_pool2(x)

      x = self.flatten(x)

      x = self.fc1(x)
      x = self.relu3(x)
      x = self.fc2(x)

      x = self.sigmoid(x)

      return x

model = ConvNueralNet()

In [50]:
model.to(device)

ConvNueralNet(
  (conv_layer1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu1): ReLU()
  (max_pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv_layer2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU()
  (max_pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=131072, out_features=64, bias=True)
  (relu3): ReLU()
  (fc2): Linear(in_features=64, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [51]:
train_dataset = TensorDataset(train_X, train_Y)
test_dataset = TensorDataset(test_X, test_Y)
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
num_epochs = 30
loss_func = nn.BCELoss()
optimizer = optim.Adam(model.parameters())

In [52]:
train_model(model,train_loader,test_loader,num_epochs,loss_func,optimizer)

Epoch 1/30, Loss: 1.0351
Test Accuracy after Epoch 1: 56.75%
Epoch 2/30, Loss: 0.6895
Test Accuracy after Epoch 2: 59.00%
Epoch 3/30, Loss: 0.6794
Test Accuracy after Epoch 3: 55.00%
Epoch 4/30, Loss: 0.6650
Test Accuracy after Epoch 4: 60.50%
Epoch 5/30, Loss: 0.6242
Test Accuracy after Epoch 5: 63.25%
Epoch 6/30, Loss: 0.6178
Test Accuracy after Epoch 6: 64.25%
Epoch 7/30, Loss: 0.5659
Test Accuracy after Epoch 7: 64.25%
Epoch 8/30, Loss: 0.5127
Test Accuracy after Epoch 8: 61.00%
Epoch 9/30, Loss: 0.4695
Test Accuracy after Epoch 9: 64.50%
Epoch 10/30, Loss: 0.4035
Test Accuracy after Epoch 10: 64.25%
Epoch 11/30, Loss: 0.3353
Test Accuracy after Epoch 11: 66.50%
Epoch 12/30, Loss: 0.2727
Test Accuracy after Epoch 12: 66.75%
Epoch 13/30, Loss: 0.2427
Test Accuracy after Epoch 13: 63.75%
Epoch 14/30, Loss: 0.1707
Test Accuracy after Epoch 14: 65.75%
Epoch 15/30, Loss: 0.1147
Test Accuracy after Epoch 15: 67.75%
Epoch 16/30, Loss: 0.0859
Test Accuracy after Epoch 16: 67.50%
Epoch 17/3

In [53]:
def train_model_with_augment(model, augment_layer, train_loader, test_loader, num_epochs, loss_function, optimizer):

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0.0
        for i, data in enumerate(train_loader):
            inputs,labels = data
            optimizer.zero_grad()
            output = model(augment_layer(inputs))
            loss = loss_function(output,labels.view(output.shape))
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        average_loss = total_loss/len(train_loader)

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {average_loss:.4f}")
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for data in test_loader:
                inputs, labels = data
                outputs = model(inputs)
                pred = (outputs > 0.5)*1
                correct += (pred==labels.view(pred.shape)).sum()
                total += labels.size(0)
            accur = 100*correct/total
            print(f"Test Accuracy after Epoch {epoch+1}: {accur:.2f}%")

    print("Training done.")

In [55]:
optimizer = optim.Adam(model.parameters())
loss_func = nn.BCELoss()
# PART 2: TODO
# Chain together transforms to create a set of allowed transformations within augment
# Available transformations are listed here: https://pytorch.org/vision/0.9/transforms.html
# Rotation, ColorJitter are popular transforms
from torchvision.transforms import v2
augment = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
])

train_model_with_augment(model, augment, train_loader, test_loader, num_epochs, loss_func, optimizer)

Epoch 1/30, Loss: 0.2685
Test Accuracy after Epoch 1: 72.00%
Epoch 2/30, Loss: 0.1917
Test Accuracy after Epoch 2: 71.00%
Epoch 3/30, Loss: 0.1711
Test Accuracy after Epoch 3: 72.25%
Epoch 4/30, Loss: 0.1697
Test Accuracy after Epoch 4: 69.75%
Epoch 5/30, Loss: 0.2009
Test Accuracy after Epoch 5: 69.50%
Epoch 6/30, Loss: 0.1467
Test Accuracy after Epoch 6: 72.50%
Epoch 7/30, Loss: 0.1300
Test Accuracy after Epoch 7: 72.50%
Epoch 8/30, Loss: 0.1428
Test Accuracy after Epoch 8: 73.25%
Epoch 9/30, Loss: 0.1129
Test Accuracy after Epoch 9: 72.50%
Epoch 10/30, Loss: 0.1283
Test Accuracy after Epoch 10: 70.00%
Epoch 11/30, Loss: 0.1353
Test Accuracy after Epoch 11: 72.75%
Epoch 12/30, Loss: 0.1446
Test Accuracy after Epoch 12: 70.50%
Epoch 13/30, Loss: 0.0971
Test Accuracy after Epoch 13: 70.25%
Epoch 14/30, Loss: 0.1198
Test Accuracy after Epoch 14: 69.00%
Epoch 15/30, Loss: 0.1291
Test Accuracy after Epoch 15: 71.50%
Epoch 16/30, Loss: 0.1067
Test Accuracy after Epoch 16: 71.75%
Epoch 17/3